# DyNet RNNs チュートリアル

チュートリアルを意訳しています。

参考
* https://github.com/clab/dynet/blob/master/examples/python/tutorials/RNNs.ipynb
* https://media.readthedocs.org/pdf/dynet/latest/dynet.pdf

In [2]:
from dynet import *

# LSTM/RNN 全体像

１層のRNNは隠れ層を次のように分けている $h_1,...,h_k$, $h_i$ は時間ごとの指標を表している
それぞれの層$h_i$は入力$x_i$と出力$r_i$に加えて過去の入力$r_{i-1}$も与える
複数層の場合は連続ではなく網目状になる。例えば

* $h_1^3,...,h_k^3$
* $h_1^2,...,h_k^2$
* $h_1^1,...h_k^1$

上記の場合は

隠れ層$h_i^1$の入力は$x_i$と$r_{i-1}^1$

隠れ層$h_i^2$の入力は$r_i^1$と$r_{i-1}^2$　など

# LSTM インターフェース

RNN / LSTM / GRU は同様のインターフェースになる。
"builder"が連続列のためのパラメータを作成する

In [3]:
model = Model()
NUM_LAYERS = 2
INPUT_DIM = 50
HIDDEN_DIM = 10
builder = LSTMBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, model)

"Builder"でRNNの内部パラメータをモデルに加えることでパラメータを意識しなくて良くなるが最適化の際は同時に行ってくれる！！

In [4]:
s0 = builder.initial_state()
s0

In [5]:
print(vecInput(10))
x1 = vecInput(INPUT_DIM)
print(x1)

expression 22/0
expression 23/0


In [6]:
s1 = s0.add_input(x1)
y1 = s1.output()

In [7]:
y1.npvalue().shape

(10,)

In [8]:
s1.h()

(expression 33/0, expression 43/0)

In [9]:
s2 = s1.add_input(x1)
y2 = s2.output()
y2

expression 67/0

１層の層で$y2$は隠れ層の状態と等しい。しかしながら最終層でもある。
隠れ層の状態にアクセスしたい場合は下記のメソッドを使用する

In [10]:
s2.h()

(expression 55/0, expression 67/0)

LSTMの状態の出力を比較してみると

In [11]:
print("all layers:", s1.h())

all layers: (expression 33/0, expression 43/0)


In [12]:
print( s1.s())

(expression 29/0, expression 39/0, expression 33/0, expression 43/0)


同様のインターフェースでシンプルなRNNも作成可能

In [13]:
rnnbuilder = SimpleRNNBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, model)

rs0 = rnnbuilder.initial_state()

rs1 = rs0.add_input(x1)
ry1 = rs1.output()
print( rs1.s())

(expression 75/0, expression 77/0)


* 出力は"output"メソッドで行う
* 隠れ層の値は"h()"メソッドで行う
* 状態は"s()"で確認する

LSTMのみ出力が４つになっている。これは入力層、忘却層、過去の入力層、出力層が存在するためである。

In [14]:
rnn_h = rs1.h()
rnn_s = rs1.s()
print("RNN h:", rnn_h)
print("RNN s:", rnn_s)

lstm_h = s1.h()
lstm_s = s1.s()
print("LSTM h:", lstm_h)
print("LSTM s:", lstm_s)

RNN h: (expression 75/0, expression 77/0)
RNN s: (expression 75/0, expression 77/0)
LSTM h: (expression 33/0, expression 43/0)
LSTM s: (expression 29/0, expression 39/0, expression 33/0, expression 43/0)


# RNNとLSTMインターフェースのオプション

Stack LSTM: 過去の状態を覚えておけるのでそこに新たに追加してグラフの作成が行える。



In [15]:
s2 = s1.add_input(x1)
s3 = s2.add_input(x1)
s4 = s3.add_input(x1)

# s3に新しい入力を加える
s5 = s3.add_input(x1)

#下記の２つの異なる列ができている
# s0, s1, s2, s3, s4
# s0, s1, s2, s3, s5

assert(s5.prev() == s3)
assert(s4.prev() == s3)

s6 = s3.prev().add_input(x1)
# s0, s1, s2, s6

In [16]:
s6.h()

(expression 185/0, expression 197/0)

4つの列が出力される

In [17]:
s6.s()

(expression 181/0, expression 193/0, expression 185/0, expression 197/0)

# メモリの効果的な変換

入力列をlist化して今までのことを一気にやってくれる手法があります。

In [18]:
state = rnnbuilder.initial_state()
xs = [x1, x1, x1]
states = state.add_inputs(xs)
outputs = [s.output() for s in states]
hs = [s.h() for s in states]
print(outputs)
print(hs)

[expression 201/0, expression 205/0, expression 209/0]
[(expression 199/0, expression 201/0), (expression 203/0, expression 205/0), (expression 207/0, expression 209/0)]


transduceを使用すればもっとメモリを効率よく使用でき、少ない行数でかけます。

In [19]:
state = rnnbuilder.initial_state()
xs = [x1, x1, x1]
outputs = state.transduce(xs)
print(outputs)

[expression 213/0, expression 217/0, expression 221/0]


# 言葉レベルのLSTM

ここから本格的な応用に入ります。言葉レベルのRNNですprint(outputs)

* モデルのパラメータの初期的設定
* 言葉の設定。
* 明示的に終了と始まり符号を表す<EOS>を追加
* コンピューターで扱うため、数字と文字の変換用の辞書を用意

In [27]:
import random
from collections import defaultdict
from itertools import count
import sys

LAYERS = 2
INPUT_DIM = 50 
HIDDEN_DIM = 50  

characters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz ,.")
characters.append("<EOS>")

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)

# モデルの設定

* modelのパラメータ設定
* 61ページにlookupパラメータの意味があります。Word Embeddingのためです。
http://phontron.com/slides/emnlp2016-dynet-tutorial-part1.pdf
* params["lookup"]は入力用
* params["R"]は出力用
* params["bias"]はバイアス用

In [28]:
model = Model()


srnn = SimpleRNNBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, model)
lstm = LSTMBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, model)

params = {}
params["lookup"] = model.add_lookup_parameters((VOCAB_SIZE, INPUT_DIM))
params["R"] = model.add_parameters((VOCAB_SIZE, HIDDEN_DIM))
params["bias"] = model.add_parameters((VOCAB_SIZE))

* 一つの文章に対してのメソッド：文章ごと長さが異なり処理を分けたいため定義
* pick関数は資料の38ページ参照
https://media.readthedocs.org/pdf/dynet/latest/dynet.pdf

In [29]:
# return compute loss of RNN for one sentence
def do_one_sentence(rnn, sentence):
    # setup the sentence
    renew_cg()
    s0 = rnn.initial_state()
    
    
    R = parameter(params["R"])
    bias = parameter(params["bias"])
    lookup = params["lookup"]
    sentence = ["<EOS>"] + list(sentence) + ["<EOS>"]
    sentence = [char2int[c] for c in sentence]
    s = s0
    loss = []
    for char,next_char in zip(sentence,sentence[1:]):
        s = s.add_input(lookup[char])
        probs = softmax(R*s.output() + bias)
        loss.append( -log(pick(probs,next_char)) )
    loss = esum(loss)
    return loss

モデルからの文字列生成

In [30]:
# generate from model:
def generate(rnn):
    def sample(probs):
        rnd = random.random()
        for i,p in enumerate(probs):
            rnd -= p
            if rnd <= 0: break
        return i
    
    # setup the sentence
    renew_cg()
    s0 = rnn.initial_state()
    
    R = parameter(params["R"])
    bias = parameter(params["bias"])
    lookup = params["lookup"]
    
    s = s0.add_input(lookup[char2int["<EOS>"]])
    out=[]
    while True:
        probs = softmax(R*s.output() + bias)
        probs = probs.vec_value()
        next_char = sample(probs)
        out.append(int2char[next_char])
        if out[-1] == "<EOS>": break
        s = s.add_input(lookup[next_char])
    return "".join(out[:-1]) # strip the <EOS>

学習のための関数

In [31]:
def train(rnn, sentence):
    trainer = SimpleSGDTrainer(model)
    for i in range(200):
        loss = do_one_sentence(rnn, sentence)
        loss_value = loss.value()
        loss.backward()
        trainer.update()
        if i % 5 == 0:
            print(loss_value)
            print(generate(rnn))

#実行例 RNN

In [34]:
sentence = "a quick brown fox jumped over the lazy dog"
train(srnn, sentence)

236.1481170654297
V    ledCecOR nNe i iJ   p,lIni
111.73921203613281
vkMjxHIOePeinYaaPeCwe lavf,ab uGGtaoQBIdlcRPerIeA iveifdPiite QaePninZMPenI t,qBIyJsrqGPea,edVKA kezu aheiaa HwuTHwKCEw. eCCNioneZ iYen iheVRHAeA azeQlaneHGjEeLaieY.lzahaveMlaIveklInQnai HIeiE,n j.IIE, phlvDiJrJMPVcI
69.71271514892578
a WyPzc dHgnekyX Cnu  wd Z KA Kot t dn,lO  gLan JJ. ,MA NdonehInZAd xucS 
39.31816864013672
a crRCi qorQuNG  roHn MQZ NrVIeriTAer oqe WaAp laIdudNg,WW mdonetAA eoPao.NZCJaoXeoeA qhez aneL .zkZeH vveA ,NeMlankbAof
18.293487548828125
a quick bzotnEGzx jumped orurgohe eaYe GqKid how
6.304571628570557
a quiCk boocn foXVj aoeci


1.4589166641235352
a quick brown fox jumped over the lazy dog
0.6376856565475464
a quick brown fox jumped over the lazy dog
0.4317273199558258
a quick brown fox jumped over the lazy dog
0.3268947899341583
a quink brown fox jumped over the lazy dog
0.26317718625068665
a quick brown fox jumped over the lazy dog
0.22029975056648254
a quicu brown fox jumped over the lazy dog
0.18946921825408936
a quick brown foxGjumped over the lazy dog
0.1662178486585617
a quick brown fox jumped over the lazy dog
0.14805704355239868
a quick brown fox jumped over the lazy dog
0.1334829330444336


a quick brown fox jumped over the lazy dog
0.12151727825403214
a quick brown fox jumped over the lazy dog
0.11152743548154831
a quick brown fox jumped over the lazy dog
0.10305308550596237
a quick brown fox jumped over the lazy dog
0.0957774892449379
a quick brown fox jumped over the lazy dog
0.0894622728228569
a quick brown fox jumped over the lazy dog
0.0839262530207634
a quick brown fox jumped over the lazy dog
0.07903673499822617
a quick brown fox jumped over the lazy dog
0.07468710094690323
a quick brown fox jumped over the lazy dog
0.0707893893122673
a quick brown fox jumped over the lazy dog
0.06727690249681473
a quick brown fox jumped over the lazy dog
0.06409820914268494


a quick brown fox jumped over the lazy dog
0.061207789927721024
a quick brown fox jumped over the lazy dog
0.05856694281101227
a quick brown fox jumped over the lazy dog
0.056139763444662094
a quick brown fox jumped over the lazy dog
0.05390631780028343
a quick brown fox jdmped over the lazy dog
0.051854994148015976
a quick brown fox jumped over the lazy dog
0.04994118958711624
a quick brown fox jumped over the lazy dog
0.04816441610455513
a quick brown fox jumped over the lazy dog
0.0465107336640358
a quick brown fox jumped over the lazy dog
0.044971633702516556
a quick brown fox jumped over the lazy dog
0.043531518429517746
a quick brown fox jumped over the lazy dog


0.04217630624771118
a quick brown fox jumped over the lazy dog
0.04090500995516777
a quick brown fox jumped over the lazy dog
0.0397111251950264
a quick brown fox jumped over the lazy dogbr dg fkuzu dog


#実行例 LSTM

In [35]:
sentence = "a quick brown fox jumped over the lazy dog"
train(lstm, sentence)

407.3851318359375
I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen Apple pen Ap ne PeAp PPAP PPAP
224.27468872070312
I have han an have A I halu han han a PiWe I IGn han hcel I havi han a pie. I have a pen I I have han abive. I have Pi I havv a pen tahe a pel. Ihhve han hive  I aan an an ave P I have hhi iAA PiAL have  pUeA hWll a I han hae a pee. I have han an an ave a Pineah hine I have I IjAI have P I have hn I have aApen I havn aFile. I have. I have a pen I IiaA hPea I have  IiMve P I hanea pee. I have aHpe. I hNvh have P I have  I have P I hale  ciAP have han han Phkve han aiI aape a pen. I have a Fi avpile apple. I haie an an an aPen I hhve han aAI hav I hale mive. hSv hav a pe a I hai. I have han a I haA I have a hile. I han a I have han have. I have han han az have a I han an an an ave a I hDve han an anea ple. I haam hive   hinea pee. I have han han a Pveu I have a pea. I have a pel. I have  plea pe., I iave P I hayv han a pele I


148.99322509765625
I hn P
111.03691101074219
I hhtAI qaroy
88.94863891601562
E taz n uLi dn dP wgoe
68.3099136352539


I havk bou qu P rmve mukpe
53.675193786621094
I dbmZ jrFocL ble Wafg dom Yoe jaokeApeq oan ja
43.33391571044922
I hao jr twe rdoiwm goe foj tued over Ian Au qwiAk bam lg
31.829288482666016
in bri
21.72420310974121
aD Euo d humy buiid, jo trgck fog duboe
15.442975997924805


a q rCwc ovei over qhe lai dudioF brown ftoeApe, Aaicpel le
9.578254699707031
han jooW
6.019927024841309
aqI quin jox ju
3.29563045501709
a quick brogd bwog
1.9137645959854126
W quick rumped over the lazy dog
1.3916668891906738


a quick brown fox jumped over the lazy dog
1.0904008150100708
a quick brown fox jumpeb ovwr tae lazy dog
0.8941783905029297
a quick brown fox jumped over the lazy dog
0.7559454441070557
a quick brown fox jumped over the lazy dog
0.6533534526824951


a quick brown fox jumped over the lazy dogd oxer the lazy dog
0.5745507478713989
a quick brown fox jumped over the lazy dog
0.5123358964920044
a quick brown fox jumped over the lazz dog
0.46204671263694763
a quick brown fox jumped over the lazy dog
0.4206182360649109
a quick brown fox jumped over the lazygdo fox jumped over the lazy dog


0.3859432339668274
a quick brown fox jumped over the lazy dog
0.3565158545970917
a quick brown fox jumped over the lazy dog
0.33122363686561584
a quick brown fox jumped over the lazy dog
0.30925658345222473
a quick brown fox jumped over the lazy dog
0.28999221324920654


a quick brown fox jumped over the lazy dog
0.2729540467262268
a quicd brown fox jumped over the lazy dog
0.257779061794281
a quick brown fox juj qu dog
0.2441854476928711
a quick brown fox jumped over the lazy dog
0.23192983865737915
a quick brown fox jumped over the lazy dog
0.22082608938217163


a quick brown fox jumped over the lazy dog
0.2107265144586563
a quick brown fox jumped over the lazy dog
0.2014971673488617
a quick brown fox jumped over the lazy dog
0.19304190576076508
a quick brown fox jumped over the lazy dog
0.1852589100599289
a quick brown fox jumped over the lazy dog
0.17806701362133026


a quick brown fox jumped over the lazy dog
0.17141544818878174
a quick brown fox jumped over the lazy dog


In [36]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

581.0768432617188
a quick brown fox jumped over mhe lazy dog
80.7076187133789
a pen b
29.160541534423828


a q qan.f bpee. Pineapple PenP PxPe
20.348590850830078
a Pineapple pan
14.379410743713379


I have a pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a Pineapple. Pineapple pan.j Pineapple pan.Apple pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a pen. I have an apple. Apple a Pineapple. Pineapple pan fPAP
11.341286659240723
I have a pen. I have a pen. I have a PineapplemAP
12.960704803466797


I have a
11.388411521911621
I have a pen. I have a pen. I Lave a Pineapple. Paneapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple pan.Apple pen, Pineapple Apple Pen PPAP PPAP PPAP PPAP PPAA PPAP PPAP PPAP PPAP PPAP PPAP PPAP PPAP
11.969993591308594


I have a PiAP
10.483719825744629
I have an apple. Apple pen. I have an apple. Apple pen. I have a pen. I have a Peneapple. Pineapple pan.Apple pen, Pineapple pan.Apple.p PPAP
18.223918914794922

...


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.9161453247070312


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.869955062866211
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.8431320190429688


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.8111820220947266
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7916518449783325


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.767293930053711


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.751880407333374
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP

In [37]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

1.7322301864624023
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7193708419799805


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I Iave a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple.. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7029553651809692


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6918668746948242
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6779470443725586


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6682463884353638
I have a pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6564781665802002


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6480209827423096
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP

...


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.5192756652832031
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.549663782119751
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


5.425973892211914
I have a pen. I have a Pineapple Apple Pen PPAP PPAP PPAP
10.695525169372559


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.468654751777649
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.2837448120117188


I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I hav

In [37]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

1.7322301864624023
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7193708419799805


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I Iave a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple.. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7029553651809692


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6918668746948242
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6779470443725586


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6682463884353638
I have a pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6564781665802002


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6480209827423096
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP

...


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.5192756652832031
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.549663782119751
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


5.425973892211914
I have a pen. I have a Pineapple Apple Pen PPAP PPAP PPAP
10.695525169372559


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.468654751777649
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.2837448120117188


I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a pen. I hav